In [1364]:
from IPython.display import HTML
display(HTML("<head><link rel='stylesheet' type='text/css' href='./../../static/custom.css'></head>"))
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1365]:
import numpy as np

import bqplot as bq
import bqplot.marks as bqm
import bqplot.scales as bqs
import bqplot.axes as bqa

import ipywidgets as widgets

In [1366]:
def get_process(vi,pi,j):
    '''
    This function calculates the (x,y) points to
    draw an polytropic curve of index j, for an ideall gas.
    
    Inputs:
    vi: float value for initial point volume
    pi: float value for initial point pressure
    j: polytopic index of the process
    
    Returns:
    v_values: 1d numpy array of len=pts containing the x values of the points
    p_values: 1d numpy array of len=pts containing the y values of the points
    '''
    
    v_values = np.linspace(v_min, v_max, pts)
    p_values = np.empty((pts))
    for i in range(pts):
        p_values[i] = pi* (vi/v_values[i])**j
    return v_values, p_values
    

In [1367]:
def get_work(vi, pi, vf, pf, v_values, p_values):
    '''
    This function calculates the work done on the system (positive work)
    or done by the system (negative work), in a process described by the set of points
    (v_value, p_values), that goes from (vi,pi) to (vf,pf). This is accomplished by numerically
    integrating the curve:
    
    $$W = -\int_{i}^{f} pdV $$
    
    Inputs:
    vi: float value for initial point volume
    pi: float value for initial point pressure
    vf: float value for final point volume
    pf: float value for final point pressure
    v_values: 1d numpy array of len=pts containing the x values of the points
    p_values: 1d numpy array of len=pts containing the y values of the points
    
    
    Returns:
    W = float value for the work done on the system (positive W) or by the system (negative W)
    '''

    W = 0.0
    dv = (v_max - v_min) / pts
    j = 0
    if vf < vi:
        vi, vf = vf, vi
        dv = -dv
    for i  in range(pts):
        v = v_values[i]
        p = p_values[i]
        if v > vf:
            break
        elif v > vi:
            W = W - p*dv
    W = C*W
    return W
        

In [1368]:
def get_energy_change(vi, pi, vf, pf, gamma):
    '''
    This function calculates the change in the energy of an ideal gas
    with adiabatic coefficien gamma tha undergoes a process starting
    on point (vi,pi) and ending on point (vf,pf)
    
    Inputs:
    vi: float value for initial point volume
    pi: float value for initial point pressure
    vf: float value for final point volume
    pf: float value for final point pressure
    gamma: adiabatic coefficient of the gas
    
    '''

    Cv = 1.0 / (gamma-1.0) #Adimensional value of Cv (Cv/NR indeed)
    dU = C * Cv*(vf*pf - vi*pi) # Energy difference in Joules
    return dU

In [1369]:
def update_pT_labels(Ti,pf,Tf):
    '''
    This function updates the labels of the pressure and
    Temperatures widgets that show as outputs on the GUI.
    This function is called whenever any of this values is changed.
    
    Inputs:
    Ti: float value for new initial temperature
    pf: float value for new final pressure
    Tf: float value for new final temperature
    '''
    
    Ti_text.value = '%.2f' % Ti
    pf_text.value ='%.2f' % pf
    Tf_text.value ='%.2f' % Tf

In [1370]:
def update_energy_labels(W, dU, Q):
    '''
    This function updates the labels of the energies
    widgets that show as outputs on the GUI.
    his function is called whenever any of this values is changed.
    
    Inputs:
    W: float value for new work
    dU: float value for new energy change
    Q: float value for new heat
    '''
    
    work_text.value = '%.0f' % W
    energy_text.value ='%.0f' % dU
    heat_text.value ='%.0f' % Q

In [1371]:
def update_fill(change):
    '''
    This function controls the filled area mark that represents
    the work done. Updates the values for defining the are whenever
    the curve changes. It also controls when to show or hide the area
    and it's color (to match the color of the current process).
    '''
    
    vi = vi_slider.value
    vf = vf_slider.value
    process = j_slider.index
    
    v_values = isocurves.x[process]
    p_values = isocurves.y[process]
    
    fill_x = []
    fill_y = []
    
    if show_work_check.value == True:
        
    
        for i in range(pts):
            v = v_values[i]
            p = p_values[i]
            if v > vi and v < vf:
                fill_x.append(v)
                fill_y.append(p)
        fillcurve.x = fill_x
        fillcurve.y = fill_y
        fillcurve.fill_opacities = [0.1]
        
    else:
        fillcurve.opacities = [0.0]
    
    fillcurve.x = fill_x
    fillcurve.y = fill_y
    newcolor = colors[process]
    fillcurve.fill_colors = [colors[process]]

In [1372]:
def hex_to_rgb(number_hex):
    
    if '#' in number_hex:
        number_hex = number_hex[1:]
    
    return (int(number_hex[0:2], 16), int(number_hex[2:4], 16), int(number_hex[4:], 16))

In [1373]:
def rgb_to_hex(number_rgb):
    
    return '#' + format(number_rgb[0], '02x') + format(number_rgb[1], '02x') + format(number_rgb[2], '02x')

In [1374]:
def generate_gradient(initial, final, length):
    
    i_r, i_g, i_b = hex_to_rgb(initial)
    f_r, f_g, f_b = hex_to_rgb(final)
    
    r_step = (f_r - i_r)/length
    g_step = (f_g - i_g)/length
    b_step = (f_b - i_b)/length
    
    r, g, b = i_r, i_g, i_b
    colors = []
    
    for i in range(length):

        h = rgb_to_hex((int(round(r)),int(round(g)),int(round(b))))
        
        colors.append(h)
        
        r = r + r_step
        g = g + g_step
        b = b + b_step

    return colors

In [1375]:
def update_labels(change):
    j_values = j_slider.options
    j_values_formatted = []
    for j in j_values:
        j_values_formatted.append('j=%.2f' % j)
    
    isocurves.labels = j_values_formatted

In [1376]:
def change_mol(change):
    '''
    This function calculates the new temperature values whenever
    the mol value is changed, and calls the update_pT_label function
    to update is outputed values. 
    '''

    vi = vi_slider.value
    pi = pi_slider.value
    vf = vf_slider.value
    gamma = gamma_dropdown.value
    j = j_slider.value
    N = mol_slider.value
    
    Ti = vi*pi/N/R
    pf = pi* (vi/vf)**j
    Tf = vf*pf/N/R
    
    update_pT_labels(Ti,pf,Tf)

In [1377]:
def update_points(change):
    '''
    This function updates the markers of the initial and final points
    and recalculates the values for temperature and energies and its
    outputed values.
    '''
    
    vi = vi_slider.value
    pi = pi_slider.value
    gamma = gamma_dropdown.value
    j = j_slider.value
    process = j_slider.index
    N = mol_slider.value
    
#    # If process is isochoric, vf must be updated to equal vi
#    if process == 0:
#        vf_slider.value=vi
    vf = vf_slider.value

    #Calculate temperature and pressure values
    Ti = vi*pi/N/R
    pf = pi * (vi/vf)**j
    Tf = vf*pf/N/R
    
    #Calculate energies
    v_values = isocurves.x[process]
    p_values = isocurves.y[process]
    W=get_work(vi,pi,vf,pf,v_values,p_values)
    dU=get_energy_change(vi,pi,vf,pf,gamma)
    Q=dU-W
    
    # Update the marks
    InitialPoint.x = [vi]
    InitialPoint.y = [pi]
    FinalPoint.x = [vf]
    FinalPoint.y = [pf]
    
    update_pT_labels(Ti,pf,Tf)
    update_energy_labels(W,dU,Q)
    
#    update_fill(None)

In [1378]:
def update_opacities(change):
    '''
    This functions updated the opacities of the curves, when the
    selected proces is changed. It then calls the update_points function
    to update the inital and final point, as the final point pressure will
    in general change when the process changes. If a isochoric process is
    selected, it changes the righ_block_122_000 layout to disable final
    volume slider and enable a new slider for final pressure. If any other
    process is selected, the default layout is displayed.
    '''
    process = j_slider.index
    j_num = jnum_slider.value
    
    # Update opacities
    opacities = []
    for i in range(j_num):
        opacities.append(0.3)
    opacities[process] = 1.0
    isocurves.opacities = opacities
    
#    # If process is isochoric, vf is fixed, so vf_slider must be disabled
#    if process == 0:
#        vf_slider.value = vi
#        vf_slider.disabled = True
#        center_block_122_000.children = [fig_122_001,vf_slider,
#                                         pf_slider,
#                                         widgets.HBox([widgets.Label(value='$T_f=$'),Tf_text,widgets.Label(value='$K$')])
#                                        ]
#    else:
#        vf_slider.disabled = False
#        center_block_122_000.children = [fig_122_001,vf_slider,
#                                         widgets.HBox([widgets.Label(value='$p_f=$'),pf_text,widgets.Label(value='$atm$')]),
#                                         widgets.HBox([widgets.Label(value='$T_f=$'),Tf_text,widgets.Label(value='$K$')])
#                                        ]
#    
#    # Update final point tracer
    update_points(None)

In [1379]:
def update_figure(change):
    '''
    This funcion recalculates the (x,y) poit sets of the four processes
    whenever the inital point is changed and replots them. Then it calls
    the update_point function to update the intial and final point marks
    and recalculate all the other parameters
    '''
    
    vi = vi_slider.value
    pi = pi_slider.value
    gamma = gamma_dropdown.value
    j_num = jnum_slider.value
    j_values = j_slider.options
    
    # Calcuate new points
    x_values = []
    y_values = []
    

    for i in range(len(j_values)):
        v_values , p_values = get_process(vi, pi, j_values[i])
        x_values.append(v_values)
        y_values.append(p_values)
    
    
    #Update the isocurves
    isocurves.x = x_values
    isocurves.y = y_values
    colors = generate_gradient('#FF0000', '#FFfa00', j_num) #generate a gradient from red to yellow
    isocurves.colors = colors
    
    
    update_opacities(None)
    update_points(None)

In [1380]:
def generate_j_values(Change):
    j_min = jmin_slider.value
    j_max = jmax_slider.value
    j_num = jnum_slider.value
    
    j_values = np.linspace(j_max, j_min, j_num)
    j_slider.options = j_values
    j_slider.value = j_min
    
    update_figure(None)
    update_labels(None)

In [1381]:
#######################
###   PARAMETERS    ###
#######################

# Global Parameters
R = 0.082057 # In atm*L/mol/K
C = 101.325 # Conversion factor from atm*L to J -> C = J/atmL
pts = 200 # Number of points for plotting each process

# Limits of parameters (volumes in L, pressures in atm)
v_min = 0.01
v_max = 20.0
p_min = 0.01
p_max = 20.0
j_lowerbound = 0.0
j_upperbound = 5.0
max_j_num = 10

## Default values
# j values
j_min = 0.0
j_max = 2.0
j_num = 4

# Gas
vi = 1.0
pi = 14.0
vf = 14.0
gamma = 5.0/3.0 # Adiabatic index (5/3 for monoatomic gases, 7/5 for diatomic gases at room temperature)
N = 1.0 # Number of mols of gas



# Default final state
vf = 14.0

# Figure parameters
colors = ['#0079c4','#f09205','#21c400','#dd4e4f']
isocurves_opacities = [0.3,0.3,1.0,0.3]


#######################
###INIT CALCULATIONS###
#######################

# Generate j slider values
j_values = np.linspace(j_max, j_min, j_num)


# Pressure and Tempetures
Ti = vi*pi/N/R
pf = pi* (vi/vf)**j
Tf = vf*pf/N/R


# Energies
W = 0.0#get_work(vi, pi, vf, pf, x_values[process], y_values[process])
dU = 0.0#get_energy_change(vi, vf, pi, pf, gamma)
Q = 0.0#dU - W






########################
###CREATE THE FIGURES###
########################

fig_122_001 = bq.Figure(title='Prozesu politropikoak',
                marks=[],
                axes=[],
                animation_duration=0,
                legend_location='top-right',
                background_style= {'fill': 'white',  'stroke': 'black'},
                fig_margin=dict(top=70, bottom=60, left=80, right=30),
                toolbar = True,
                layout=widgets.Layout(width='100%')
    )


scale_x = bqs.LinearScale(min = v_min, max = v_max)
scale_y = bqs.LinearScale(min = p_min, max = p_max)

axis_x = bqa.Axis(scale=scale_x,
                tick_format='.1f',#'0.2f',
                tick_style={'font-size': '15px'},
                tick_values = np.linspace(v_min, v_max, 11),
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                label='v',
                label_location='middle',
                label_style={'stroke': 'black', 'default-size': 35},
                label_offset='50px')

axis_y = bqa.Axis(
                scale=scale_y,
                tick_format='.1f',#'0.2f',
                tick_style={'font-size': '15px'},
                tick_values= np.linspace(p_min, p_max, 6),
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                orientation='vertical',
                label='p (atm)',
                label_location='middle',
                label_style={'stroke': 'red', 'default_size': 35},
                label_offset='50px')

fig_122_001.axes = [axis_x,axis_y]


########################
####CREATE THE MARKS####
########################


isocurves = bqm.Lines(
                x = [], 
                y = [], 
                scales = {'x': scale_x, 'y': scale_y},
                colors = colors,
                opacities = isocurves_opacities,
                #fill_colors = ['green'],
                #fill_opacities = [0.0],
                #fill = 'bottom',
                labels = ['j=0.0', 'j=0.5', 'j=1.0', 'Adiabatikoa'],
                display_legend=True
)



InitialPoint = bqm.Scatter(
    name = 'Initial Point',
    x = [vi],
    y = [pi],
    scales = {'x': scale_x, 'y': scale_y}, 
    #opacities = [1.0],
    visible = True,
    colors = ['green'],
    names = [],
    labels=['Hasierako egoera'],
    display_legend = True
)      


FinalPoint = bqm.Scatter(
    name = 'Final Point',
    x = [vf],
    y = [pf],
    scales = {'x': scale_x, 'y': scale_y}, 
    #opacities = [1.0],
    visible = True,
    colors = ['blue'],
    names = [],
    labels=['Bukaerako egoera'],
    display_legend = True
)      

fillcurve = bqm.Lines(
                x = [], 
                y = [], 
                scales = {'x': scale_x, 'y': scale_y},
                colors = colors,
                opacities = [0.0],
                fill_colors = [colors[0]],
                fill_opacities = [0.0],
                fill = 'bottom',
                labels = ['Lana'],
                display_legend=False
)

fig_122_001.marks = [isocurves, InitialPoint, FinalPoint, fillcurve]

########################
######  WIDGETS  #######
########################


## Top block (j index values generator block)

jmin_slider = widgets.FloatSlider(
    value=j_min,
    min=j_lowerbound,
    max=j_upperbound,
    step=0.1,
    description='$j_{min}$',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout=widgets.Layout(width='100%'),
)

jmax_slider = widgets.FloatSlider(
    value=j_max,
    min=j_lowerbound,
    max=j_upperbound,
    step=0.1,
    description='$j_{max}$',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout=widgets.Layout(width='100%'),
)

jnum_slider = widgets.IntSlider(
    value=j_num,
    min=1,
    max=max_j_num,
    #step=1,
    description='$j_{num}$',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    #readout_format='.1f',
    layout=widgets.Layout(width='100%'),
)

generate_button = widgets.Button(
    description='Generate',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    layout=widgets.Layout(width='100%'),
    tooltip='Click me'
)

generate_button.on_click(generate_j_values)


## Left block (Initial state and process block)

gamma_dropdown = widgets.Dropdown(
    options=[('Monoatomikoa',5.0/3.0), ('Diatomikoa',7.0/5.0)],
    value=gamma,
    description='Gasa',
    disabled=False,
    layout=widgets.Layout(width='100%')
)

gamma_dropdown.observe(update_figure, 'value')

mol_slider = widgets.FloatSlider(
    value=1.0,
    min=v_min,
    max=v_max,
    step=0.1,
    description='$N$',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout=widgets.Layout(width='100%'),
)

mol_slider.observe(change_mol, 'value')



vi_slider = widgets.FloatSlider(
    value=vi,
    min=v_min,
    max=v_max,
    step=0.1,
    description='$v_i$',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout=widgets.Layout(width='100%'),
)

vi_slider.observe(update_figure, 'value')


pi_slider = widgets.FloatSlider(
    value=pi,
    min=p_min,
    max=p_max,
    step=0.1,
    description='$p_i$',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout=widgets.Layout(width='100%'),
)

pi_slider.observe(update_figure, 'value')

Ti_text = widgets.Label(value='%.2f' % Ti)


j_slider = widgets.SelectionSlider(
    options=j_values,
    #value=T_values[int(T_values.size/2)],
    description=r'\( j \)',
    disabled=False,
    continuous_update=True,
    orientation='vertical',
    readout=True,
    layout = widgets.Layout(width = '%10', height = '80%', margin = '45px 0 0 0')
)

j_slider.observe(update_opacities, 'value')
## Center block (figure block)

vf_slider = widgets.FloatSlider(
    value=vf,
    min=v_min,
    max=v_max,
    step=0.1,
    description='$v_f$',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout=widgets.Layout(width='100%'),
)

vf_slider.observe(update_points, 'value')

pf_text = widgets.Label(value='%.2f' % pf)
Tf_text = widgets.Label(value='%.2f' % Tf)

pf_slider = widgets.FloatSlider(
    value=pf,
    min=p_min,
    max=p_max,
    step=0.1,
    description='$p_f$',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout=widgets.Layout(width='100%'),
)

pf_slider.observe(update_points, 'value')


## Right block (energy block)

work_text = widgets.Label(value='%.0f' % W)
energy_text = widgets.Label(value='%.0f' % dU)
heat_text = widgets.Label(value='%.0f' % Q)

show_work_check = widgets.Checkbox(
    description='Erakutsi lana',
    disabled=True,
    value=False,
)

show_work_check.observe(update_fill,'value')


########################
###  INIT FIGURES  ####
########################

update_figure(None)




########################
######  LAYOUT  ########
########################

## Top Block ##
top_block_122_000 = widgets.HBox([], layout=widgets.Layout(width='100%', align_items='center'))
top_block_122_000.children = [jmin_slider, jmax_slider, jnum_slider, generate_button]


## Left Block ##
left_block_122_000 = widgets.VBox([], layout=widgets.Layout(width='20%', align_items='center'))
left_block_122_000.children = [gamma_dropdown, mol_slider, widgets.Label(value="Hasierako egoera:"), vi_slider, pi_slider,
                               widgets.HBox([widgets.Label(value='$T_i=$'),Ti_text,widgets.Label(value='$K$')]),
                               ]
                              
## Center Block ##
center_block_122_000 = widgets.VBox([], layout=widgets.Layout(width='65%', align_items='center'))
center_block_122_000.children = [widgets.HBox([j_slider,fig_122_001], layout=widgets.Layout(width='100%', align_items='center')),
                                 vf_slider,
                                 widgets.HBox([widgets.Label(value='$p_f=$'),pf_text,widgets.Label(value='$atm$')]),
                                 widgets.HBox([widgets.Label(value='$T_f=$'),Tf_text,widgets.Label(value='$K$')])
                                ]

## Right Block ##
right_block_122_000 = widgets.VBox([], layout=widgets.Layout(width='15%'))
right_block_122_000.children = [
                                widgets.HBox([widgets.Label(value='$W=$'),work_text,widgets.Label(value='$J$')]),
                                widgets.HBox([widgets.Label(value='$\Delta U=$'),energy_text,widgets.Label(value='$J$')]),
                                widgets.HBox([widgets.Label(value='$Q=$'),heat_text,widgets.Label(value='$J$')]),
                                show_work_check
                               ]
                               
                                

## Main Block ##

body_block_122_000 = widgets.HBox([],layout=widgets.Layout(width='100%', align_items='center'))
body_block_122_000.children = [left_block_122_000, center_block_122_000,right_block_122_000]

main_block_122_000 = widgets.VBox([],layout=widgets.Layout(width='100%', align_items='center'))
main_block_122_000.children = [top_block_122_000, body_block_122_000]

main_block_122_000

IndexError: index 6 is out of bounds for axis 0 with size 4

IndexError: index 5 is out of bounds for axis 0 with size 4

IndexError: index 4 is out of bounds for axis 0 with size 4

IndexError: index 4 is out of bounds for axis 0 with size 4

IndexError: index 5 is out of bounds for axis 0 with size 4

...

IndexError: index 6 is out of bounds for axis 0 with size 4

IndexError: index 8 is out of bounds for axis 0 with size 7

IndexError: index 7 is out of bounds for axis 0 with size 7

IndexError: index 7 is out of bounds for axis 0 with size 7

IndexError: index 7 is out of bounds for axis 0 with size 7